In [1]:
import csv
import random
import polars as pl
import pandas as pd
import sys
import os
from pydantic import BaseModel
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from csv_validator.validator import PolarsValidator, PolarsReader, SlowerIterValidator, CsvReader, IterValidator, PandasReader, PandasValidator

def create_benchmark():
    with open("benchmark_1m.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["a", "b", "c"])
        for i in range(1, 1000001):
            a = i
            b = f"val{i}"
            c = round(random.uniform(1.0, 5.0), 2)
            writer.writerow([a, b, c])

class BenchmarkSchema(BaseModel):
    a: int
    b: str
    c: float

In [6]:
create_benchmark()

In [8]:
filename = "benchmark_1m.csv"
pl_reader = PolarsReader(pl.read_csv(filename))
pd_reader = PandasReader(pd.read_csv(filename))
csv_reader = CsvReader(filename)

In [9]:
%%timeit
validator = PandasValidator(BenchmarkSchema, pd_reader)
validator.validate_all()

5.46 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
validator = PolarsValidator(BenchmarkSchema, pl_reader)
validator.validate_all()

/Users/thomasscelsi/Documents/programming/codex/src/csv_validator/validator.py:63: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = self.reader.read().with_columns(
/Users/thomasscelsi/Documents/programming/codex/src/csv_validator/validator.py:63: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = self.reader.read().with_columns(
/Users/thomasscelsi/Documents/programming/codex/src/csv_validator/validator.py:63: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = self.reader.read().with_columns(
/Users/thomasscelsi/Documents/programming/codex/src/csv_validator/validator.py:63: MapWithoutReturnDtypeWarning: Calling 

1.68 s ± 8.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
validator = SlowerIterValidator(BenchmarkSchema, csv_reader)
validator.validate_all()

1.37 s ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
validator = IterValidator(BenchmarkSchema, csv_reader)
validator.validate_all()

1.22 s ± 8.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
